For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is it's performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


### There are twelve different authors in the gutenberg corpus. These will be used for the project

In [3]:
# Load the text files
emma = gutenberg.raw('austen-emma.txt')
persuasion = gutenberg.raw('austen-persuasion.txt')
sense = gutenberg.raw('austen-sense.txt')
bible = gutenberg.raw('bible-kjv.txt')
poems = gutenberg.raw('blake-poems.txt')
stories = gutenberg.raw('bryant-stories.txt')
busterbrown = gutenberg.raw('burgess-busterbrown.txt')
alice = gutenberg.raw('carroll-alice.txt')
ball = gutenberg.raw('chesterton-ball.txt')
brown = gutenberg.raw('chesterton-brown.txt')
thursday = gutenberg.raw('chesterton-thursday.txt')
parents = gutenberg.raw('edgeworth-parents.txt')
moby_dick = gutenberg.raw('melville-moby_dick.txt')
paradise = gutenberg.raw('milton-paradise.txt')
caesar = gutenberg.raw('shakespeare-caesar.txt')
hamlet = gutenberg.raw('shakespeare-hamlet.txt')
macbeth = gutenberg.raw('shakespeare-macbeth.txt')
leaves = gutenberg.raw('whitman-leaves.txt')

In [4]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text


# The Chapter indicator is idiosyncratic
emma = re.sub(r'Chapter \d+', '', emma)
persuasion = re.sub(r'Chapter \d+', '', persuasion)
sense = re.sub(r'Chapter \d+', '', sense)
bible = re.sub(r'Chapter \d+', '', bible)
poems = re.sub(r'Chapter \d+', '', poems)
stories = re.sub(r'Chapter \d+', '', stories)
busterbrown = re.sub(r'Chapter \d+', '', busterbrown)
alice = re.sub(r'Chapter \d+', '', alice)
ball = re.sub(r'Chapter \d+', '', ball)
brown = re.sub(r'Chapter \d+', '', brown)
thursday = re.sub(r'Chapter \d+', '', thursday)
parents = re.sub(r'Chapter \d+', '', parents)
moby_dick = re.sub(r'Chapter \d+', '', moby_dick)
paradise = re.sub(r'Chapter \d+', '', paradise)
caesar = re.sub(r'Chapter \d+', '', caesar)
hamlet = re.sub(r'Chapter \d+', '', hamlet)
macbeth = re.sub(r'Chapter \d+', '', macbeth)
leaves = re.sub(r'Chapter \d+', '', leaves)

emma = text_cleaner(emma)
persuasion = text_cleaner(persuasion)
sense = text_cleaner(sense)
bible = text_cleaner(bible)
poems = text_cleaner(poems)
stories = text_cleaner(stories)
busterbrown = text_cleaner(busterbrown)
alice = text_cleaner(alice)
ball = text_cleaner(ball)
brown = text_cleaner(brown)
thursday = text_cleaner(thursday)
parents = text_cleaner(parents)
moby_dick = text_cleaner(moby_dick)
paradise = text_cleaner(paradise)
caesar = text_cleaner(caesar)
hamlet = text_cleaner(hamlet)
macbeth = text_cleaner(macbeth)
leaves = text_cleaner(leaves)

In [27]:
# Checking to make sure the texts look correct:
#print(emma)
#print(persuasion)
#print(sense)
#print(bible[:1000])
#print(poems)
#print(stories)
#print(busterbrown)
#print(alice)
#print(ball)
#print(brown)
#print(thursday)
#print(parents)
#print(moby_dick)
#print(paradise)
#print(caesar)
#print(hamlet)
#print(macbeth)
#print(leaves)
# The cleaner was successful for each of the texts.

In [28]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
emma_doc = nlp(emma)
persuasion_doc = nlp(persuasion)
sense_doc = nlp(sense)
bible_doc = nlp(bible)
poems_doc = nlp(poems)
stories_doc = nlp(stories)
busterbrown_doc = nlp(busterbrown)
alice_doc = nlp(alice)
ball_doc = nlp(ball)
brown_doc = nlp(brown)
thursday_doc = nlp(thursday)
parents_doc = nlp(parents)
moby_dick_doc = nlp(moby_dick)
paradise_doc = nlp(paradise)
caesar_doc = nlp(caesar)
hamlet_doc = nlp(hamlet)
macbeth_doc = nlp(macbeth)
leaves_doc = nlp(leaves)

In [32]:
# Group into sentences.
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
sense_sents = [[sent, "Austen"] for sent in sense_doc.sents]
bible_sents = [[sent, "KJV"] for sent in bible_doc.sents]
poems_sents = [[sent, "Blake"] for sent in poems_doc.sents]
stories_sents = [[sent, "Bryant"] for sent in stories_doc.sents]
busterbrown_sents = [[sent, "Burgess"] for sent in busterbrown_doc.sents]
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
ball_sents = [[sent, "Chesterton"] for sent in ball_doc.sents]
brown_sents = [[sent, "Chesterton"] for sent in brown_doc.sents]
thursday_sents = [[sent, "Chesterton"] for sent in thursday_doc.sents]
parents_sents = [[sent, "Edgeworth"] for sent in parents_doc.sents]
moby_dick_sents = [[sent, "Melville"] for sent in moby_dick_doc.sents]
paradise_sents = [[sent, "Milton"] for sent in paradise_doc.sents]
caesar_sents = [[sent, "Shakespeare"] for sent in caesar_doc.sents]
hamlet_sents = [[sent, "Shakespeare"] for sent in caesar_doc.sents]
macbeth_sents = [[sent, "Shakespeare"] for sent in macbeth_doc.sents]
leaves_sents = [[sent, "Whitman"] for sent in leaves_doc.sents]

# Combine the sentences into one data frame.
sentences = pd.DataFrame(emma_sents + persuasion_sents + sense_sents + bible_sents
                         + poems_sents + stories_sents + busterbrown_sents + 
                         alice_sents + ball_sents + brown_sents + thursday_sents
                         + parents_sents + moby_dick_sents + paradise_sents + 
                         caesar_sents + hamlet_sents + macbeth_sents + leaves_sents)
sentences.head()

,0,1
0,"(VOLUME, I, CHAPTER, I, Emma, Woodhouse, ,, ha...",Austen
1,"(She, was, the, youngest, of, the, two, daught...",Austen
2,"(Her, mother, had, died, too, long, ago, for, ...",Austen
3,"(Sixteen, years, had, Miss, Taylor, been, in, ...",Austen
4,"(Between, _, them, _, it, was, more, the, inti...",Austen


In [36]:
################################################################################
################ RESERVE 25% OF YOUR CORPUS AS A TEST SET ######################
################################################################################

from sklearn.model_selection import train_test_split

Y = sentences[1]
X = sentences[0]

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.25,
                                                    random_state=0)

### The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

### Since the number of clusters is unknown, affinity propogation will be used. Estimated number of clusters will be pulled from the affinity model.

In [1]:
################################################################################
########## NEED TO DO BOW BEFORE CLUSTERING ####################################
################################################################################

In [34]:
#############################################################################
### This throws ValueError: setting an array element with a sequence. #######
### Do I need to use tf-idf vectorizer before I can use clustering? #########

from sklearn.cluster import AffinityPropagation
from sklearn import metrics

# Declare the model and fit it in one statement.
# Note that you can provide arguments to the model, but we didn't.
af = AffinityPropagation().fit(X_train)
print('Done')

# Pull the number of clusters and cluster assignments for each data point.
cluster_centers_indices = af.cluster_centers_indices_
n_clusters_ = len(cluster_centers_indices)
labels = af.labels_

print('Estimated number of clusters: {}'.format(n_clusters_))

ValueError: setting an array element with a sequence.

### Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. 

### Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

### Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is it's performance consistent?

### If there is a divergence in the relative stability of your model and your clusters, delve into why.

### Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!